In [ ]:
# Install required packages
!pip install -q kaggle
!pip install -q pyspark
!pip install -q xgboost
!pip install -q category_encoders
!pip install -q imblearn
!pip install -q tensorflow keras

import os
from google.colab import files

# Upload kaggle.json for Kaggle API authentication
uploaded = files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Instacart dataset using Kaggle API
!kaggle datasets download -d yasserh/instacart-online-grocery-basket-analysis-dataset -p /content/instacart_dataset --unzip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.1 MB/s eta 0:00:00


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset
License(s): CC0-1.0
 72% 142M/197M [00:00<00:00, 1.48GB/s]
100% 197M/197M [00:00<00:00, 833MB/s] 


In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session
spark = SparkSession.builder \
        .appName("Instacart Market Basket Analysis") \
        .config("spark.sql.shuffle.partitions", "8") \
        .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [ ]:
data_path = '/content/instacart_dataset/'

aisles = spark.read.csv(data_path + 'aisles.csv', header=True, inferSchema=True)
departments = spark.read.csv(data_path + 'departments.csv', header=True, inferSchema=True)
orders = spark.read.csv(data_path + 'orders.csv', header=True, inferSchema=True)
order_products_prior = spark.read.csv(data_path + 'order_products__prior.csv', header=True, inferSchema=True)
order_products_train = spark.read.csv(data_path + 'order_products__train.csv', header=True, inferSchema=True)
products = spark.read.csv(data_path + 'products.csv', header=True, inferSchema=True)

# Cache frequently used DataFrames for performance
orders.cache()
order_products_prior.cache()
products.cache()

# Show sample data
aisles.show(3)
departments.show(3)
orders.show(3)
products.show(3)


+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
+--------+--------------------+
only showing top 3 rows

+-------------+----------+
|department_id|department|
+-------------+----------+
|            1|    frozen|
|            2|     other|
|            3|    bakery|
+-------------+----------+
only showing top 3 rows

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  NULL|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|         

In [ ]:
# Create temporary views for SQL queries
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
orders.createOrReplaceTempView("orders")
order_products_prior.createOrReplaceTempView("order_products_prior")
products.createOrReplaceTempView("products")

# Example: Join tables for analysis - e.g. order_products_prior with products and aisles
query = """
SELECT
    opp.order_id,
    opp.product_id,
    p.product_name,
    a.aisle,
    d.department,
    opp.add_to_cart_order,
    opp.reordered,
    o.user_id,
    o.order_number,
    o.order_dow,
    o.order_hour_of_day,
    o.days_since_prior_order
FROM order_products_prior opp
JOIN products p ON opp.product_id = p.product_id
JOIN aisles a ON p.aisle_id = a.aisle_id
JOIN departments d ON p.department_id = d.department_id
JOIN orders o ON opp.order_id = o.order_id
"""

instacart_full = spark.sql(query)
instacart_full.createOrReplaceTempView("instacart_full")
instacart_full.show(5)


+--------+----------+--------------------+----------------+----------+-----------------+---------+-------+------------+---------+-----------------+----------------------+
|order_id|product_id|        product_name|           aisle|department|add_to_cart_order|reordered|user_id|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+----------+--------------------+----------------+----------+-----------------+---------+-------+------------+---------+-----------------+----------------------+
|      12|     30597|French Vanilla Co...|           cream|dairy eggs|                1|        1| 152610|          22|        6|                8|                  10.0|
|      12|     15221|             2% Milk|            milk|dairy eggs|                2|        1| 152610|          22|        6|                8|                  10.0|
|      12|     43772|Cherubs Heavenly ...|fresh vegetables|   produce|                3|        1| 152610|          22|        6|                

In [ ]:
# ============================================================
# 🧠 Optimized Temporal Market Basket Analysis (PySpark)
# ============================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from itertools import combinations
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# ============================================================
# 1️⃣ Spark Session with Increased Memory
# ============================================================
spark = (
    SparkSession.builder
    .appName("Instacart_Temporal_Support_Optimized")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.shuffle.partitions", "100")
    .config("spark.sql.execution.arrow.enabled", "true")
    .getOrCreate()
)

data_path = '/content/instacart_dataset/'

aisles = spark.read.csv(data_path + 'aisles.csv', header=True, inferSchema=True)
departments = spark.read.csv(data_path + 'departments.csv', header=True, inferSchema=True)
orders = spark.read.csv(data_path + 'orders.csv', header=True, inferSchema=True)
order_products_prior = spark.read.csv(data_path + 'order_products__prior.csv', header=True, inferSchema=True)
order_products_train = spark.read.csv(data_path + 'order_products__train.csv', header=True, inferSchema=True)
products = spark.read.csv(data_path + 'products.csv', header=True, inferSchema=True)

order_products = order_products_prior.union(order_products_train)

# Limit for safe experimentation (you can increase gradually)
orders_sample = orders.sample(0.15, seed=42)  # Use 15% of orders
order_products = order_products.join(orders_sample.select("order_id"), "order_id")

# ============================================================
# 2️⃣ Join & Select Relevant Columns Only
# ============================================================
instacart_full = (
    order_products
    .join(F.broadcast(products), "product_id")
    .join(F.broadcast(aisles), "aisle_id")
    .join(F.broadcast(departments), "department_id")
    .join(orders_sample, "order_id")
    .select("order_id", "user_id", "order_dow", "order_number", "product_name")
)

# ============================================================
# 3️⃣ Aggregate Items per Order
# ============================================================
order_items = (
    instacart_full
    .groupBy("order_id", "user_id", "order_dow", "order_number")
    .agg(F.collect_set("product_name").alias("items"))
    .filter(F.size("items") > 1)
)

# ============================================================
# 4️⃣ Generate Item Pairs per Order
# ============================================================
def get_pairs(item_list):
    from itertools import combinations
    return [{"A": a, "B": b} for a, b in combinations(sorted(item_list), 2)]

pair_schema = ArrayType(
    StructType([StructField("A", StringType()), StructField("B", StringType())])
)
get_pairs_udf = F.udf(get_pairs, pair_schema)

pairs_df = (
    order_items
    .withColumn("pairs", get_pairs_udf("items"))
    .withColumn("pair", F.explode("pairs"))
    .select("user_id", "order_id", "order_number", "order_dow",
            F.col("pair.A").alias("A"), F.col("pair.B").alias("B"))
)

# ============================================================
# 5️⃣ Temporal Support (per Day-of-Week)
# ============================================================
temporal_support = (
    pairs_df
    .groupBy("A", "B", "order_dow")
    .agg(F.countDistinct("order_id").alias("pair_count"))
    .withColumn("temporal_support",
                F.col("pair_count") / F.sum("pair_count").over(Window.partitionBy("order_dow")))
)

# Top 10 pairs per day
top_temporal = (
    temporal_support
    .withColumn("rank", F.row_number().over(Window.partitionBy("order_dow").orderBy(F.desc("temporal_support"))))
    .filter(F.col("rank") <= 10)
)

print("=== ✅ Temporal Support (Top 10 Pairs per Day-of-Week) ===")
top_temporal.show(30, truncate=False)

# ============================================================
# 6️⃣ Reorder Stability
# ============================================================
reorder_stability = (
    pairs_df
    .groupBy("user_id", "A", "B")
    .agg(F.countDistinct("order_id").alias("pair_orders"))
    .groupBy("A", "B")
    .agg(F.avg("pair_orders").alias("avg_reorder_stability"))
    .orderBy(F.desc("avg_reorder_stability"))
)

print("\n=== ✅ Reorder Stability (Top 10 Persistent Pairs) ===")
reorder_stability.show(10, truncate=False)

# ============================================================
# 7️⃣ Sequential Confidence
# ============================================================
w = Window.partitionBy("user_id").orderBy("order_number")

user_sequences = (
    instacart_full
    .withColumn("next_product", F.lead("product_name").over(w))
    .select("user_id", "product_name", "next_product")
    .na.drop()
)

seq_conf = (
    user_sequences
    .groupBy("product_name", "next_product")
    .agg(F.count("*").alias("count_AB"))
    .withColumnRenamed("product_name", "A")
    .withColumnRenamed("next_product", "B")
)

total_A = (
    user_sequences
    .groupBy("product_name")
    .agg(F.count("*").alias("count_A"))
    .withColumnRenamed("product_name", "A")
)

seq_conf = (
    seq_conf
    .join(total_A, "A")
    .withColumn("sequential_confidence", F.col("count_AB") / F.col("count_A"))
    .orderBy(F.desc("sequential_confidence"))
)

print("\n=== ✅ Sequential Confidence (Top 10 Temporal Sequences) ===")
seq_conf.show(10, truncate=False)

# ============================================================
# ✅ Summary
# ============================================================
print("""
✅ Metrics Computed Successfully:
1. Temporal Support — product-pair frequency by day-of-week.
2. Reorder Stability — how consistently users reorder pairs.
3. Sequential Confidence — likelihood that B follows A chronologically.
""")


=== ✅ Temporal Support (Top 10 Pairs per Day-of-Week) ===
+----------------------+--------------------+---------+----------+---------------------+----+
|A                     |B                   |order_dow|pair_count|temporal_support     |rank|
+----------------------+--------------------+---------+----------+---------------------+----+
|Bag of Organic Bananas|Organic Strawberries|4        |1089      |2.6339103991571486E-4|1   |
|Bag of Organic Bananas|Organic Hass Avocado|4        |1026      |2.481535417387727E-4 |2   |
|Banana                |Organic Strawberries|4        |982       |2.3751147951995593E-4|3   |
|Banana                |Organic Avocado     |4        |871       |2.1066445892248636E-4|4   |
|Bag of Organic Bananas|Organic Baby Spinach|4        |827       |2.000223967036696E-4 |5   |
|Banana                |Organic Baby Spinach|4        |805       |1.9470136559426122E-4|6   |
|Organic Hass Avocado  |Organic Strawberries|4        |753       |1.8212438297202324E-4|7   |
|B

In [ ]:
# Example: Total orders by day of week
spark.sql("""
SELECT order_dow, COUNT(DISTINCT order_id) as total_orders
FROM orders
GROUP BY order_dow
ORDER BY order_dow
""").show()


+---------+------------+
|order_dow|total_orders|
+---------+------------+
|        0|      600905|
|        1|      587478|
|        2|      467260|
|        3|      436972|
|        4|      426339|
|        5|      453368|
|        6|      448761|
+---------+------------+

